In [ ]:
#import libraires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn import metrics
from tensorflow.keras import layers, callbacks
from tensorflow import keras

In [ ]:
#read csv file
df = pd.read_csv("hotel_bookings.csv")

In [ ]:
#Check dataset
print("Shape:",df.shape)
print("Number of duplicated rows:",df.duplicated().sum())
print("Number of Null value:\n",df.isnull().sum())

In [ ]:
#drop duplicated rows
df=df.drop_duplicates()

In [ ]:
#fill agent and company columns wiht 0
fill= {"agent": 0, "company": 0}
df= df.fillna(fill)

#drop children and country columns
df.drop('children',axis=1, inplace=True)
df.drop('country',axis=1, inplace=True)

In [ ]:
#descriptive statistics
df.describe()

In [ ]:
#summary of a DataFrame
df.info()

In [ ]:
q1= df['adr'].quantile(0.25)
q3= df['adr'].quantile(0.75)
IQR=q3-q1

IQR

lower=q1-1.5*IQR
upper=q3+1.5*IQR

df= df[(df['adr'] >= lower) & (df['adr'] <= upper)]

In [ ]:
# select all categorical variables
df_categorical = df.select_dtypes(include=['object'])

In [ ]:
# apply Label encoder to df_categorical
enc= LabelEncoder()
df_categorical = df_categorical.apply(enc.fit_transform)
df.drop(df_categorical.columns, axis=1, inplace=True)
df = pd.concat([df, df_categorical], axis=1)

In [ ]:
feature=['lead_time', 'adr', 'required_car_parking_spaces',
       'total_of_special_requests',
       'market_segment', 'distribution_channel','deposit_type']

In [ ]:
#drop unnecessary columns with droping reservation_status
drop_cols=[
 'arrival_date_year',
 'arrival_date_month',
 'arrival_date_week_number',
 'arrival_date_day_of_month',
 'agent',
 'company',
 'reservation_status',
 'is_canceled']

In [ ]:
# Define features and target variable
X=df.drop(drop_cols,axis=1)
y=df['is_canceled']

In [ ]:
# Split data into training and testing sets (70% train, 30% test)
X_train,X_test, y_train, y_test= train_test_split(X, y, test_size=.3, random_state=42)

In [ ]:
# feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# build network model
model = keras.Sequential([
    layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
# compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy']
)

In [ ]:
# apply early stopping 
early_stop = callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)

In [ ]:
# train the model 
history = model.fit(
    X_train_scaled, y_train,
    validation_split=.2,
    batch_size=512,
    epochs=100,
    callbacks=[early_stop],
)

In [ ]:
#compare the model loss against the actual val_loss
history_df = pd.DataFrame(history.history)
history_df[['loss', 'val_loss']].plot(title="Cross-Entropy Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.show()

#compare the model accuracy against the actual val accuracy
history_df[['binary_accuracy', 'val_binary_accuracy']].plot(title="Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.grid(True)
plt.show()

In [ ]:
# confusion matrix
y_pred = model.predict(X_test_scaled)
y_pred_labels = (y_pred > 0.5).astype(int)

cm = metrics.confusion_matrix(y_test, y_pred_labels)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Calculate performance metrics
acc = metrics.accuracy_score(y_test, y_pred_labels)
prec = metrics.precision_score(y_test, y_pred_labels)
rec = metrics.recall_score(y_test, y_pred_labels)
f1 = metrics.f1_score(y_test, y_pred_labels)

print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall: {rec:.4f}")
print(f"F1 Score: {f1:.4f}")

In [ ]:
# test evaluate
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)
print(f"Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")